In [8]:
# Importing Libraries & Functions
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import re
import warnings
warnings.filterwarnings('ignore')
from IPython.lib.display import Audio
framerate = 4410
# play_time_seconds = 3
# t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
# audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
# Audio(audio_data, rate=framerate, autoplay=True)

#SciKit Learn 
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, mean_absolute_error
from sklearn.metrics import accuracy_score, auc, roc_curve, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

import xgboost as xgb
from xgboost import XGBClassifier

def outputs(cm):
    acc = np.round((cm.ravel()[0]+cm.ravel()[3])/sum(cm.ravel()),4)
    tpr = np.round(cm.ravel()[3] / (cm.ravel()[3] + cm.ravel()[2]),4)
    fpr = np.round(cm.ravel()[1] / (cm.ravel()[1] + cm.ravel()[0]),4)
    outputs = [acc, tpr, fpr]
    return outputs

## Loading Data + Train/Test Split

First, we load in the two CSVs from the FP1_CUM_STATS and FP2_XFACTOR notebooks and join them on the 'GAME_ID' as the key. We then set our training data to be all data from the 2010 Season to the 2016 Season, and our testing data to be all data from the 2017 Season to the 2018 Season. The cell below performs this task, and outputs the Counts and Percentages of Training and Testing data.

In [2]:
# Splitting Data
FP1 = pd.read_csv('FP1.csv')
FP2 = pd.read_csv('FP2.csv')
df = pd.merge(FP1, FP2, on = 'GAME_ID').rename(columns={'SEASON_y':'SEASON'})
df = df.drop(['TIMESTAMP','HOME_TEAM_NAME'], axis=1)

train_df = df[df['SEASON'].between(2007,2014)]
val_df = df[df['SEASON'].between(2015,2016)]
test_df = df[df['SEASON'].between(2017,2018)]

train_df, val_df, test_df = train_df.drop(['SEASON', 'GAME_ID'], axis = 1), val_df.drop(['SEASON', 'GAME_ID'], axis = 1), test_df.drop(['SEASON', 'GAME_ID'], axis = 1)
X_train, y_train = train_df.drop('HOME_TEAM_WINS', axis = 1), train_df['HOME_TEAM_WINS']
X_val, y_val = val_df.drop('HOME_TEAM_WINS', axis = 1), val_df['HOME_TEAM_WINS']
X_test, y_test = test_df.drop('HOME_TEAM_WINS', axis = 1), test_df['HOME_TEAM_WINS']

train_prop = np.round(len(X_train) / (len(df)), 3)
val_prop = np.round(len(X_val) / (len(df)), 3)
test_prop = np.round(len(X_test) / (len(df)), 3)

print('Training Data: ' + str(len(X_train)) + ' rows -- ' + str(np.round(train_prop*100,2)) + '%')
print('Validation Data: ' + str(len(X_val)) + ' rows -- ' + str(np.round(val_prop*100,2)) + '%')
print('Testing Data: ' + str(len(X_test)) + ' rows -- ' + str(np.round(test_prop*100,2)) + '%')

Training Data: 10216 rows -- 66.1%
Validation Data: 2625 rows -- 17.0%
Testing Data: 2624 rows -- 17.0%


## Model \#0: Baseline (Dummy) Model

Our dummy model predicts the most frequent label in the training set, which is HOME_TEAM_WINS = 1. This is the same as always predicting the home team to win.

In [3]:
# Baseline Model 
model_0 = DummyClassifier(strategy = "most_frequent")
model_0.fit(X_train, y_train)
y_pred_0 = model_0.predict(X_test)

# Confusion Matrix & Outputs 
cm_0 = confusion_matrix(y_test, y_pred_0)
outputs_0 = outputs(cm_0)
print ("\nConfusion Matrix : \n", cm_0) 
print ("\nAccuracy : ", outputs_0[0])


Confusion Matrix : 
 [[   0 1079]
 [   0 1545]]

Accuracy :  0.5888


## PCA for Feature Reduction


In [178]:
# # Implementing PCA

# # Fitting Scaler to X_train 
# scaler = StandardScaler()
# scaler.fit(X_train)

# # Apply transform to both the training set and the test set.
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

# # Make an instance of the Model ==> 0.95 means 95% of variance explained
# pca = PCA(n_components = 0.999999999999999)
# pca.fit(X_train, y_train)
# print('Number of Components: ', pca.n_components_)

# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)

In [203]:
# # Scree Plots
# yy = pca.explained_variance_ratio_
# xx = np.arange(1,len(yy)+1)

# sns.set_style('darkgrid')
# fig, ax = plt.subplots(figsize = (12,6))

# plt.plot(xx, yy)
# plt.xlabel('PCA #', fontsize = 16), plt.ylabel('Explained Variance %', fontsize = 16);

## Model \#1: Logistic Regression

The cell below cretes a Logistic Regression model and calculates the outputs to be presented later.

In [4]:
# Logistic Regression

model_1 = LogisticRegression(random_state = 69, tol = 0.00001, verbose = 3, n_jobs = -1)
model_1.fit(X_train, y_train)
y_prob_1 = model_1.predict_proba(X_test)
y_pred_1 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_1[:,1]], index = y_test.index)

# Confusion Matrix & Outputs 
cm_1 = confusion_matrix(y_test, y_pred_1)
outputs_1 = outputs(cm_1)
print ("\nConfusion Matrix : \n", cm_1) 
print ("\nAccuracy : ", outputs_1[0]) 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.



Confusion Matrix : 
 [[ 470  609]
 [ 275 1270]]

Accuracy :  0.6631


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.9s finished


In [ ]:
# cm_df = pd.DataFrame(cm_1).rename(columns = {0: 'TRUE HOME WINS', 1: 'TRUE AWAY WINS'})
# cm_df = cm_df.rename({0: 'PREDICT HOME WINS', 1: 'PREDICT AWAY WINS'})
# print(cm_df.to_latex())

# out_df = pd.DataFrame(outputs_1).rename({0:'Accuracy', 1:'TPR', 2:'FPR'})
# out_df = out_df.rename(columns = {0:'Test Set Statistic'})
# print(out_df.to_latex())

## Model \#2: Random Forest 

This cell below is a Random Forest model with cross-validation on various parameters.

In [193]:
# Random Forest

tic = time.time()

model_2_acc_list = []
max_feature_list = np.linspace(1, len(X_train.columns), 100, dtype = 'int32')

for i in max_feature_list: 
    
    model_2 = RandomForestClassifier(max_features = i, n_estimators = 250, random_state = 69, 
                                     n_jobs = -1, min_samples_leaf = 5)
    model_2.fit(X_train, y_train)
   
    y_prob_2 = model_2.predict_proba(X_val)
    y_pred_2 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_2[:,1]], index = y_val.index)
    
    # Confusion Matrix & Outputs 
    cm_2 = confusion_matrix(y_val, y_pred_2)
    accuracy = outputs(cm_2)[0]
    
    model_2_acc_list.append(accuracy)
    print(f'Max Features = {str(i)} -- {time.time()-tic}')
    
opt_max_features = max_feature_list[np.argmax(model_2_acc_list)]
    
print('Optimal Max Features: ', opt_max_features)
print('Optimal Validation Accuracy: ', np.max(model_2_acc_list))

toc = time.time()
minutes, seconds = np.floor((toc-tic)/60), np.round((toc-tic) - (60*minutes),3)
print(f'Elapsed Time:  {int(minutes)} min  {seconds} sec\n')

Max Features = 1 -- 0.5802397727966309
Max Features = 9 -- 1.8410797119140625
Max Features = 17 -- 3.8025858402252197
Max Features = 26 -- 6.743449687957764
Max Features = 34 -- 10.980271816253662
Max Features = 43 -- 16.78975796699524
Max Features = 51 -- 23.835094928741455
Max Features = 59 -- 32.094812870025635
Max Features = 68 -- 41.57308578491211
Max Features = 76 -- 52.20741581916809
Max Features = 85 -- 64.02583980560303
Max Features = 93 -- 77.18632769584656
Max Features = 101 -- 91.23670768737793
Max Features = 110 -- 106.66675090789795
Max Features = 118 -- 123.14095282554626
Max Features = 127 -- 141.00074696540833
Max Features = 135 -- 159.91439580917358
Max Features = 143 -- 180.0077838897705
Max Features = 152 -- 201.1083059310913
Max Features = 160 -- 223.48351573944092
Max Features = 169 -- 246.80750060081482
Max Features = 177 -- 271.87873101234436
Max Features = 185 -- 298.6088969707489
Max Features = 194 -- 326.1380808353424
Max Features = 202 -- 354.6783139705658
M

In [194]:
# Run our Best Model 

model_2 = RandomForestClassifier(max_features = opt_max_features, n_estimators = 200, random_state = 69, 
                                 n_jobs = -1, min_samples_split = 10, min_samples_leaf = 10)
model_2.fit(X_train, y_train)

y_prob_2 = model_2.predict_proba(X_test)
y_pred_2 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_2[:,1]], index = y_test.index)
# 
# Confusion Matrix & Outputs 
cm_2 = confusion_matrix(y_test, y_pred_2)
outputs_2 = outputs(cm_2)
print ("\nConfusion Matrix : \n", cm_2) 
print ("\nAccuracy : ", outputs_2[0])


Confusion Matrix : 
 [[ 370  709]
 [ 216 1329]]

Accuracy :  0.6475


In [ ]:
# max_features_RF = RF_model.cv_results_['param_max_features'].data
# acc_scores_RF = RF_model.cv_results_['mean_test_score']

# sns.set_style('darkgrid')
# plt.figure(figsize=(8, 6))
# plt.title('\nMean Cross-Validation Accuracy vs. max_features\n', fontsize=16)
# plt.xlabel('max_features', fontsize=16)
# plt.ylabel('Mean Cross-Validation Accuracy', fontsize=16)
# plt.plot(max_features_RF, acc_scores_RF, linewidth=3, color='orange')
# plt.scatter(max_features_RF[acc_scores_RF.argmax(axis=0)], np.max(acc_scores_RF), s=125, marker='o')
# plt.grid(True, which='both')
# plt.show()

## Model \#3: XGBoost

The cells below go through an entire process of creating and cross-validating an XGBoost model. First, we set default parameters and using the XGB Cross-Validation function to determine the optimal number of trees for the specified learning rate. We then use this number of estimators to cross-validate and select an approximate values for *max_depth* and *min_child_weight*, which we then find more optimally by shrinking the search space.

In [195]:
# Combining train and val

train_df_3 = df[df['SEASON'].between(2007,2016)]

train_df_3 = train_df_3.drop(['SEASON', 'GAME_ID'], axis = 1)
X_train_3, y_train_3 = train_df_3.drop('HOME_TEAM_WINS', axis = 1), train_df_3['HOME_TEAM_WINS']

In [196]:
# Defining the Cross-Validation Function 
def modelfit(alg, train, predictors, useTrainCV = True, cv_folds = 5, early_stopping_rounds = 50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(train[predictors].values, label=train[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold = cv_folds,
                          metrics = 'error', early_stopping_rounds = early_stopping_rounds, verbose_eval = True)
        alg.set_params(n_estimators = cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(train[predictors], train['HOME_TEAM_WINS'], eval_metric = 'error')
        
    #Predict training set:
    train_predictions = alg.predict(train[predictors])
    train_predprob = alg.predict_proba(train[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(train['HOME_TEAM_WINS'].values, train_predictions))
    print("AUC Score (Train): %f" % roc_auc_score(train['HOME_TEAM_WINS'], train_predprob))

Run the first cell below and the final number it outputs in square brackets is the n_estimators that you plug into gridsearchcv object below.

STEP \#1 : Calculate n_estimators for learning_rate = 0.1 (VERY HIGH)

STEP \#2 : Use Cross-Validation to tune the other hyperparameters

STEP \#3 : Now reduce the learning rate to 0.01 or something smaller and calculate the new n_estimators for the hyperparameters found above. Obviously, this will mean much more trees are used.

STEP \#4 : Run our final model using the learning rate set above, the hyperparameters found using cross-validation, and the corresponding (now higher) n_estimators.

In [197]:
# Determining Optimal Number of Estimators
target = 'HOME_TEAM_WINS'
train = pd.concat([X_train_3, y_train_3], axis = 1)
predictors = [x for x in train.columns if x not in [target]]

model_3 = XGBClassifier(learning_rate = 0.1, n_estimators = 1000, max_depth = 5, min_child_weight = 1,
                     gamma = 0, subsample = 0.8, colsample_bytree = 0.8, objective = 'binary:logistic',
                     nthread = 4, scale_pos_weight = 1, seed = 69)
modelfit(model_3, train, X_train.columns, useTrainCV = True)

[0]	train-error:0.30743+0.00701	test-error:0.34211+0.00763
[1]	train-error:0.29597+0.00441	test-error:0.33541+0.01080
[2]	train-error:0.28857+0.00521	test-error:0.33284+0.00777
[3]	train-error:0.28421+0.00481	test-error:0.33043+0.00546
[4]	train-error:0.28035+0.00391	test-error:0.32786+0.00736
[5]	train-error:0.27778+0.00277	test-error:0.32934+0.00575
[6]	train-error:0.27632+0.00419	test-error:0.32871+0.00749
[7]	train-error:0.27459+0.00516	test-error:0.32614+0.00784
[8]	train-error:0.27217+0.00425	test-error:0.32404+0.01012
[9]	train-error:0.26955+0.00450	test-error:0.32326+0.00956
[10]	train-error:0.26778+0.00359	test-error:0.32256+0.00879
[11]	train-error:0.26598+0.00419	test-error:0.32311+0.00988
[12]	train-error:0.26474+0.00307	test-error:0.32069+0.00922
[13]	train-error:0.26308+0.00306	test-error:0.32030+0.00915
[14]	train-error:0.26020+0.00282	test-error:0.31952+0.00832
[15]	train-error:0.25859+0.00295	test-error:0.32100+0.00865
[16]	train-error:0.25621+0.00329	test-error:0.3197

In [198]:
# Cross-Validating MAX_DEPTH and MIN_CHILD_WEIGHT

param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=82, gamma=0, 
                                                  subsample=0.8, colsample_bytree=0.8, 
                                                  objective= 'binary:logistic', nthread=4, 
                                                  scale_pos_weight=1, seed=27), 
                        param_grid = param_test1, 
                        scoring = 'accuracy', n_jobs = -1, iid=False , cv=5, verbose = 3)
gsearch1.fit(train[predictors],train[target])
print('Best Approx. Parameters: \n', gsearch1.best_params_, gsearch1.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  60 | elapsed:  6.3min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  6.4min finished


[22:05:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best Approx. Parameters: 
 {'max_depth': 3, 'min_child_weight': 3} 0.6634234989674395


In [201]:
# Reducing Search Space
param_test2 = {
 'max_depth':[2,3,4],
 'min_child_weight':[2,3,4]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=82, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=1,seed=27), 
                        param_grid = param_test2, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose = 3)
gsearch2.fit(train[predictors],train[target])
print('Best Parameters: \n', gsearch2.best_params_, gsearch2.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  45 | elapsed:  1.3min remaining:   39.1s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  2.3min finished


[22:33:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best Parameters: 
 {'max_depth': 2, 'min_child_weight': 3} 0.6674737069953398


We then tune *gamma* in the same way and then calculate the new optimal number of parameters for the new set of hyperparameters. We then tune *subsample*, *colsample_bytree*, and *reg_alpha* in the same way as the rest with the new optimal estimators.

In [202]:
# Cross-Validating GAMMA
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=82, max_depth=2, 
                                                  min_child_weight=3, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, 
                                                  scale_pos_weight=1,seed=27), 
                        param_grid = param_test3, scoring='accuracy',n_jobs=-1,iid=False, cv=5, verbose = 3)
gsearch3.fit(train[predictors],train[target])
gsearch3.best_params_, gsearch3.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  25 | elapsed:   28.9s remaining:  3.5min
[Parallel(n_jobs=-1)]: Done  12 out of  25 | elapsed:   29.1s remaining:   31.5s
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:   46.1s remaining:    8.8s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   46.2s finished


[22:34:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'gamma': 0.0}, 0.6674737069953398)

In [203]:
# Determining NEW Optimal Number of Estimators
xgb2 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=2, min_child_weight=3,
                     gamma=0.0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',
                     nthread=4, scale_pos_weight=1,seed=27)
modelfit(xgb2, train, predictors, useTrainCV = True)

[0]	train-error:0.34569+0.00356	test-error:0.35021+0.00850
[1]	train-error:0.34242+0.00348	test-error:0.34694+0.01157
[2]	train-error:0.33590+0.00651	test-error:0.34257+0.00894
[3]	train-error:0.33235+0.00343	test-error:0.33525+0.01369
[4]	train-error:0.32994+0.00409	test-error:0.33596+0.00954
[5]	train-error:0.32719+0.00269	test-error:0.33222+0.01084
[6]	train-error:0.32605+0.00258	test-error:0.33191+0.00894
[7]	train-error:0.32540+0.00152	test-error:0.32973+0.01075
[8]	train-error:0.32404+0.00173	test-error:0.32972+0.00819
[9]	train-error:0.32313+0.00142	test-error:0.32902+0.00993
[10]	train-error:0.32112+0.00161	test-error:0.32887+0.00861
[11]	train-error:0.31925+0.00157	test-error:0.32832+0.00851
[12]	train-error:0.31976+0.00107	test-error:0.32677+0.00939
[13]	train-error:0.31915+0.00109	test-error:0.32568+0.00929
[14]	train-error:0.31799+0.00118	test-error:0.32474+0.00996
[15]	train-error:0.31691+0.00177	test-error:0.32451+0.00963
[16]	train-error:0.31713+0.00189	test-error:0.3249

[137]	train-error:0.27566+0.00253	test-error:0.31298+0.00764
[138]	train-error:0.27496+0.00269	test-error:0.31267+0.00797
[139]	train-error:0.27465+0.00276	test-error:0.31283+0.00789
[140]	train-error:0.27414+0.00230	test-error:0.31314+0.00769
[141]	train-error:0.27428+0.00258	test-error:0.31329+0.00850
[142]	train-error:0.27397+0.00239	test-error:0.31275+0.00816
[143]	train-error:0.27385+0.00226	test-error:0.31275+0.00819
[144]	train-error:0.27377+0.00233	test-error:0.31306+0.00823
[145]	train-error:0.27367+0.00231	test-error:0.31228+0.00764
[146]	train-error:0.27317+0.00223	test-error:0.31189+0.00739
[147]	train-error:0.27303+0.00233	test-error:0.31166+0.00810
[148]	train-error:0.27276+0.00261	test-error:0.31283+0.00804
[149]	train-error:0.27278+0.00227	test-error:0.31314+0.00747
[150]	train-error:0.27243+0.00245	test-error:0.31189+0.00803
[151]	train-error:0.27245+0.00274	test-error:0.31267+0.00813
[152]	train-error:0.27229+0.00230	test-error:0.31314+0.00767
[153]	train-error:0.2722

[272]	train-error:0.24614+0.00301	test-error:0.31275+0.01144
[273]	train-error:0.24593+0.00294	test-error:0.31228+0.01207
[274]	train-error:0.24574+0.00274	test-error:0.31228+0.01196
[275]	train-error:0.24529+0.00269	test-error:0.31251+0.01136
[276]	train-error:0.24541+0.00288	test-error:0.31236+0.01172
[277]	train-error:0.24517+0.00282	test-error:0.31259+0.01145
[278]	train-error:0.24484+0.00265	test-error:0.31267+0.01200
[279]	train-error:0.24504+0.00245	test-error:0.31259+0.01170
[280]	train-error:0.24504+0.00265	test-error:0.31283+0.01185
[281]	train-error:0.24455+0.00259	test-error:0.31251+0.01183
[282]	train-error:0.24441+0.00266	test-error:0.31298+0.01039

Model Report
Accuracy : 0.7371
AUC Score (Train): 0.806295


In [204]:
# Cross-Validating SUBSAMPLE and COLSAMPLE_BYTREE
param_test4 = {
 'subsample':[i/10.0 for i in range(7,9)],
 'colsample_bytree':[i/10.0 for i in range(7,9)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=282, max_depth=2, 
                                                  min_child_weight=3, gamma=0.0, objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=1,seed=27), 
                        param_grid = param_test4, scoring='accuracy',n_jobs=-1,iid=False, cv=5, verbose = 3)
gsearch4.fit(train[predictors],train[target])
gsearch4.best_params_, gsearch4.best_score_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  20 | elapsed:  1.6min remaining:  9.1min
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed:  1.7min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  2.3min remaining:   24.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.3min finished


[22:46:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'colsample_bytree': 0.7, 'subsample': 0.7}, 0.6591398886071529)

In [205]:
#Cross-Validating REG_ALPHA
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=282, max_depth=2, 
                                                  min_child_weight=3, gamma=0.0, subsample=0.7, 
                                                  colsample_bytree=0.7, objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=1,seed=27), 
                        param_grid = param_test6, scoring='accuracy',n_jobs=-1,iid=False, cv=5, verbose = 3)
gsearch6.fit(train[predictors],train[target])
gsearch6.best_params_, gsearch6.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  25 | elapsed:  1.5min remaining: 11.3min
[Parallel(n_jobs=-1)]: Done  12 out of  25 | elapsed:  1.6min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  2.3min remaining:   26.6s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.4min finished


[22:50:18] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'reg_alpha': 100}, 0.6756480636003925)

In [206]:
#Cross-Validating REG_ALPHA
param_test6 = {
 'reg_alpha':[1, 2, 5, 10, 25, 50, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=282, max_depth=2, 
                                                  min_child_weight=3, gamma=0.0, subsample=0.7, 
                                                  colsample_bytree=0.7, objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=1,seed=27), 
                        param_grid = param_test6, scoring='accuracy',n_jobs=-1,iid=False, cv=5, verbose = 3)
gsearch6.fit(train[predictors],train[target])
gsearch6.best_params_, gsearch6.best_score_

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  35 | elapsed:  1.5min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed:  3.2min remaining:   47.5s
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:  3.4min finished


[22:55:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'reg_alpha': 100}, 0.6756480636003925)

We then calculate the optimal number of trees for our final set of hyperparameters, but decrease the learning rate to 0.01 to get our final hyperparameters and run this model.

In [207]:
# Determining Optimal Number of Trees w/ New (LOWER) Learning Rate 

xgb3 = XGBClassifier(learning_rate = 0.005, n_estimators = 5000, max_depth = 2, min_child_weight = 3, gamma = 0.0, 
                     subsample = 0.7, colsample_bytree = 0.7, reg_alpha = 100, objective = 'binary:logistic', 
                     nthread=4, scale_pos_weight = 1, seed = 69)
modelfit(xgb3, train, predictors, useTrainCV = True)

[0]	train-error:0.35611+0.00759	test-error:0.36321+0.01368
[1]	train-error:0.34729+0.00540	test-error:0.35208+0.01187
[2]	train-error:0.34178+0.00800	test-error:0.34725+0.01427
[3]	train-error:0.33975+0.00624	test-error:0.34655+0.01388
[4]	train-error:0.33732+0.00264	test-error:0.34530+0.01364
[5]	train-error:0.33599+0.00301	test-error:0.34522+0.01395
[6]	train-error:0.33529+0.00270	test-error:0.34491+0.01446
[7]	train-error:0.33337+0.00169	test-error:0.34172+0.01755
[8]	train-error:0.33292+0.00135	test-error:0.34141+0.01555
[9]	train-error:0.33202+0.00135	test-error:0.34156+0.01610
[10]	train-error:0.33220+0.00138	test-error:0.34180+0.01615
[11]	train-error:0.33226+0.00094	test-error:0.34125+0.01588
[12]	train-error:0.33165+0.00111	test-error:0.34071+0.01417
[13]	train-error:0.33173+0.00129	test-error:0.33907+0.01395
[14]	train-error:0.33144+0.00177	test-error:0.34008+0.01154
[15]	train-error:0.33167+0.00140	test-error:0.33876+0.01034
[16]	train-error:0.33173+0.00155	test-error:0.3387

In [208]:
# Running our Final XgBoost Model
xgb5 = XGBClassifier(learning_rate = 0.005, n_estimators = 120, max_depth = 2, min_child_weight = 3, gamma = 0.0,
                     subsample = 0.7, colsample_bytree = 0.7, reg_alpha = 100, objective = 'binary:logistic',
                     nthread = 4, scale_pos_weight = 1, seed = 69)

model_3 = xgb5
model_3.fit(X_train_3, y_train_3)
y_prob_3 = model_3.predict_proba(X_test)
y_pred_3 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_3[:,1]], index = y_test.index)

# Confusion Matrix & Outputs 
cm_3 = confusion_matrix(y_test, y_pred_3)
outputs_3 = outputs(cm_3)
print ("\nConfusion Matrix : \n", cm_3) 
print ("\nAccuracy : ", outputs_3[0]) 

[22:58:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

Confusion Matrix : 
 [[ 497  582]
 [ 316 1229]]

Accuracy :  0.6578


## Boosting (round2)

In [117]:
# Running our Final XgBoost Model
grid = {'n_estimators': [500, 1000, 2500], 
        'max_leaf_nodes': [4, 5, 6]}

model_3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.005, objective= 'binary:logistic', seed=6969), 
                        param_grid = grid, scoring='accuracy', n_jobs=-1, iid=False, cv=3, verbose = 3)
model_3.fit(train[predictors],train[target])
print('Optimal Hyperparameters: ', model_3.best_params_, model_3.best_score_)



y_prob_3 = model_3.best_estimator_.predict_proba(X_test)
y_pred_3 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_3[:,1]], index = y_test.index)

# Confusion Matrix & Outputs 
cm_3 = confusion_matrix(y_test, y_pred_3)
outputs_3 = outputs(cm_3)
print ("\nConfusion Matrix : \n", cm_3) 
print ("\nAccuracy : ", outputs_3[0]) 

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  27 | elapsed: 10.8min remaining: 37.9min
[Parallel(n_jobs=-1)]: Done  16 out of  27 | elapsed: 31.0min remaining: 21.3min
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 50.2min finished


[17:16:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "max_leaf_nodes" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:16:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Optimal Hyperparameters:  {'max_leaf_nodes': 4, 'n_estimators': 1000} 0.6676366672208885

Confusion Matrix : 
 [[ 511  568]
 [ 334 1211]]

Accuracy :  0.6562


In [209]:
# # Running our Final XgBoost Model
# grid = {'n_estimators': [485], 
#         'max_leaf_nodes': [2,3,4]}

# model_3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.005, objective= 'binary:logistic', seed=6969), 
#                         param_grid = grid, scoring='accuracy', n_jobs=-1, iid=False, cv=3, verbose = 3)
# model_3.fit(train[predictors],train[target])
# print('Optimal Hyperparameters: ', model_3.best_params_, model_3.best_score_)



# y_prob_3 = model_3.best_estimator_.predict_proba(X_test)
# y_pred_3 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_3[:,1]], index = y_test.index)

# # Confusion Matrix & Outputs 
# cm_3 = confusion_matrix(y_test, y_pred_3)
# outputs_3 = outputs(cm_3)
# print ("\nConfusion Matrix : \n", cm_3) 
# print ("\nAccuracy : ", outputs_3[0]) 

## Model \#4: Neural Network

The cells below go through an entire process of creating and cross-validating our neural network model using Sci-Kit Learn's MLPClassifier.

In [233]:
# Neural Network 
from sklearn.neural_network import MLPClassifier
# model_3 = MLPClassifier(hidden_layer_sizes =(75,), max_iter = 200,
#                         activation = 'relu', solver = 'adam', random_state = 69)
# model_3.fit(X_train, y_train)
# y_prob_3 = model_3.predict_proba(X_test)
# y_pred_3 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_3[:,1]], index = y_test.index)

# grid = {'hidden_layer_sizes': [(50,),(100,),(200,),(400,),(800,),(50,50),(100,50)]}

model_4 = MLPClassifier(max_iter = 100, alpha = 0.0001, activation = 'relu', hidden_layer_sizes=(25,), 
                        solver = 'adam', random_state = 69)
# model_4 = GridSearchCV(model_4, grid, n_jobs = -1, cv = 2, verbose = 3)
model_4.fit(X_train, y_train)

y_prob_4 = model_4.predict_proba(X_test)
# print(model_4.best_params_)
# y_pred_4 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_4[:,1]])
y_pred_3 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_4[:,1]], index = y_test.index)
# Confusion Matrix & Outputs 
cm_4 = confusion_matrix(y_test, y_pred_4)
outputs_4 = outputs(cm_4)
print ("\nConfusion Matrix : \n", cm_4) 
print ("\nAccuracy : ", outputs_4[0]) 


Confusion Matrix : 
 [[ 588  491]
 [ 429 1116]]

Accuracy :  0.6494


# Blending Model

In [220]:
# # Reloading the data

# # Splitting Data
# FP1 = pd.read_csv('FP1.csv')
# FP2 = pd.read_csv('FP2.csv')
# df = pd.merge(FP1, FP2, on = 'GAME_ID')
# df = df.drop(['TIMESTAMP','HOME_TEAM_NAME'], axis=1)

# train_df = df[df['SEASON'].between(2010,2016)]
# val_df = df[df['SEASON'] == 2017]
# test_df = df[df['SEASON'] == 2018]


# train_df, val_df, test_df = train_df.drop(['SEASON', 'GAME_ID'], axis = 1), val_df.drop(['SEASON', 'GAME_ID'], axis = 1), test_df.drop(['SEASON', 'GAME_ID'], axis = 1)
# X_train, y_train = train_df.drop('HOME_TEAM_WINS', axis = 1), train_df['HOME_TEAM_WINS']
# X_val, y_val = val_df.drop('HOME_TEAM_WINS', axis = 1), val_df['HOME_TEAM_WINS']
# X_test, y_test = test_df.drop('HOME_TEAM_WINS', axis = 1), test_df['HOME_TEAM_WINS']

# train_prop = np.round(len(X_train) / (len(df)), 3)
# val_prop = np.round(len(X_val) / (len(df)), 3)
# test_prop = np.round(len(X_test) / (len(df)), 3)

# print('Training Data: ' + str(len(X_train)) + ' rows -- ' + str(np.round(train_prop*100,2)) + '%')
# print('Validation Data: ' + str(len(X_val)) + ' rows -- ' + str(np.round(val_prop*100,2)) + '%')
# print('Testing Data: ' + str(len(X_test)) + ' rows -- ' + str(np.round(test_prop*100,2)) + '%')

In [231]:
y_prob_4

array([1, 1, 1, ..., 1, 1, 1])

In [234]:
# Creating blend_df 
y_prob_1_val = model_1.predict_proba(X_val)
y_prob_2_val = model_2.predict_proba(X_val)
y_prob_3_val = model_3.predict_proba(X_val)
y_prob_4_val = model_4.predict_proba(X_val)



val_df_blend = pd.DataFrame({'Logistic Regression': list(y_prob_1_val[:,1]), 
                             'Random Forest': list(y_prob_2_val[:,1]), 
                             'XGBoost': list(y_prob_3_val[:,1]), 
                             'Neural Network': y_prob_4_val[:,1], 
                             'HOME_TEAM_WINS': y_val})
test_df_blend = pd.DataFrame({'Logistic Regression': y_prob_1[:,1], 
                             'Random Forest': y_prob_2[:,1], 
                             'XGBoost': y_prob_3[:,1], 
                             'Neural Network': y_prob_4[:,1], 
                             'HOME_TEAM_WINS': y_test})
X_val_blend, y_val_blend = val_df_blend.drop('HOME_TEAM_WINS', axis=1), val_df_blend['HOME_TEAM_WINS']
X_test_blend, y_test_blend = test_df_blend.drop('HOME_TEAM_WINS', axis=1), test_df_blend['HOME_TEAM_WINS']


In [240]:
# Blended Logistic Regression

model_5 = LogisticRegression(random_state = 69,  verbose = 3, n_jobs = -1)
model_5.fit(X_val_blend, y_val_blend)
y_prob_5 = model_5.predict_proba(X_test_blend)
y_pred_5 = pd.Series([1 if x > 0.5 else 0 for x in y_prob_5[:,1]], index = y_test_blend.index)

# Confusion Matrix & Outputs 
cm_5 = confusion_matrix(y_test_blend, y_pred_5)
outputs_5 = outputs(cm_5)
print ("\nConfusion Matrix : \n", cm_5) 
print ("\nAccuracy : ", outputs_5[0]) 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.



Confusion Matrix : 
 [[ 490  589]
 [ 289 1256]]

Accuracy :  0.6654


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished


## Final Model Results

The cell below calculates the Accuracy (our primary performance metric), TPR, and FPR for each of the four models above.

In [ ]:
# # Generating the Output Table
# df = pd.DataFrame({'Baseline Model': np.round(outputs_0,4),
#                   'Logistic Regression': np.round(outputs_1,4),
#                   'Random Forest': np.round(outputs_2,4),
#                   'XGBoost': np.round(outputs_3,4),
#                   'Neural Network': np.round(outputs_4,4)}, index = ['Accuracy', 'TPR', 'FPR'], ).T
# df

In [ ]:
grid_values = {'n_estimators': np.linspace(100, 5000, 5, dtype='int32'),  # np.logspace(6, 12, num=7, base=2, dtype='int32'),
               'learning_rate': [0.01],
               'max_leaf_nodes': np.linspace(2, 10, 8, dtype='int32'),
               'max_depth': [20],
               'min_samples_leaf': [15],
               'random_state': [88]} 



gbc = GradientBoostingClassifier()
gbc_cv = GridSearchCV(gbc, param_grid=grid_values, cv=5)
gbc_cv.fit(X_train, y_train)

toc = time.time()

print('time:', round(toc-tic, 2),'s')

# Chris XGBoosting Cross-Validation

In [6]:
# Combining train and val

train_df_3 = df[df['SEASON'].between(2007,2016)]

train_df_3 = train_df_3.drop(['SEASON', 'GAME_ID'], axis = 1)
X_train_3, y_train_3 = train_df_3.drop('HOME_TEAM_WINS', axis = 1), train_df_3['HOME_TEAM_WINS']

In [ ]:
## Model Training ##
from sklearn.ensemble import GradientBoostingClassifier

tic = time.time()

grid_values_boost = {'n_estimators': [10, 50, 100, 250, 500, 1000, 2000, 4000, 5000], 
#                      'n_estimators': np.linspace(1000, 10000, 10, dtype='int32'),
                     'learning_rate': [0.005],
                     'max_leaf_nodes': [2, 4, 6, 8],
                     'max_depth': [20],
                     'min_samples_leaf':[15], 
                     'random_state': [69]}    

boost = GradientBoostingClassifier()
# cv = KFold(n_splits=5,random_state=333,shuffle=False) 
boost_cv = GridSearchCV(boost, param_grid=grid_values_boost, 
                        scoring='accuracy', cv=5, verbose=2, n_jobs=-1)
boost_cv.fit(X_train_3,y_train_3)
toc = time.time()
print('Time: '+str((toc-tic)/60)+' min')

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.1min
